In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../../..")

import lib.ehr.example_datasets.mimiciv_aki as m4aki


In [9]:
from lib.metric.metrics import  LeadingAKIPredictionAccuracy, LeadingPredictionAccuracyConfig
from lib.metric.metrics import LeadPredictionLossMetric,ObsPredictionLossMetric, PerColumnObsPredictionLoss
from lib.ml.evaluation import  EvaluationConfig


In [2]:
metrics = [
    LeadingAKIPredictionAccuracy(config=LeadingPredictionAccuracyConfig(aki_binary_index=81)),
    PerColumnObsPredictionLoss(),
    LeadPredictionLossMetric(),
    ObsPredictionLossMetric()
]

In [12]:
import lib.utils as U
eval_config = EvaluationConfig(metrics = [m.export_module() for m in metrics])
U.write_config(eval_config.to_dict(), 'eval_config.json')

In [3]:
from lib.ml.embeddings import InICENODEEmbeddingsConfig, InterventionsEmbeddingsConfig
from lib.ml.in_models import ICENODEConfig, GRUODEBayes
from lib.ml.model import Precomputes


In [4]:
from lib.ehr.tvx_transformations import TrainingSplitGroups

emb_config = InICENODEEmbeddingsConfig(dx_codes=50, demographic=10,
                                       interventions=InterventionsEmbeddingsConfig(icu_inputs=10, icu_procedures=10,
                                                                                   hosp_procedures=10,
                                                                                   interventions=20))

model_config = ICENODEConfig(state=50, lead_predictor='monotonic')

tvx0 = m4aki.TVxAKIMIMICIVDataset.load('tvx_aki_training_groups/tvx_aki_0.h5')

In [8]:
tvx0.dataset.scheme.obs.codes.index('renal_aki.aki_binary')

In [9]:
tvx0.dataset.scheme.obs.codes

In [3]:

model = GRUODEBayes.from_tvx_ehr(tvx_ehr=tvx0, config=model_config, embeddings_config=emb_config)

tvx00 = TrainingSplitGroups.subset(tvx0, tuple(tvx0.subjects.keys())[:10])
predictions = model.batch_predict(tvx00)
predictions.to_cpu().save('predictions_tx00.h5')

In [4]:
from lib.ml.artefacts import AdmissionsPrediction
predictions = AdmissionsPrediction.load('predictions_tx00.h5')

In [5]:
predictions.predictions[0].prediction